# Projet TSA

### Imports

In [ ]:
import msicpe
print(msicpe.__version__)

from msicpe import tsa
from math import *
import cv2
import numpy as np
import pandas as pd
from plotly import express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import struct
from scipy import signal
from scipy.fftpack import fft
from scipy.linalg import toeplitz

*On rappelle que la documentation de la bibliothèque msicpe peut être trouvée en ligne --> __[msicpe](https://cpe.pages.in2p3.fr/msi/toolbox/msicpe.tsa.html)__.*

## Séance 1 - MDFB

### Chargement des données


In [ ]:
Fs, s = msicpe.tsa.load_signal("ProtestMonoBruitTronque.wav")

In [ ]:
# propriétés du signal
N = len(s) # nombre de points du signal s
D = N/Fs # durée du signal s

# vecteur temps
ts = np.linspace(0, D, N)

In [ ]:
# affichage du signal s
df_s = pd.DataFrame({'x': ts , 'y': s , 'legende':'s'})
fig1 = px.line(df_s, x='x',  y='y', color='legende', labels={'x':'Temps (s)', 'y':''}, title='Signal temporel', width=700 )
fig1.show()

### Binarisation

In [ ]:
# signal binarisé
dtype = 'int8'
sb = msicpe.tsa.data2bin(s[:10], dtype)

# vecteur temps correspondant
d_bin = 10#1.25 # frequence d'echantillonnage du signal binaire
tb = np.linspace(0, len(sb)/d_bin, len(sb))

In [ ]:
# Affichage de s_b 
df_sb = pd.DataFrame({'x': tb , 'y': sb , 'legende': 'signal' })
fig2 = px.line(df_sb,x='x',  y='y', color='legende', markers="*", labels={'x': 'tb' , 'y':'sb'}, title= 'Signal binarisé' , width=700 )
fig2.show()

### Modulation

In [ ]:
# paramètres de modulation
A   = 5
nu0 = 20
nu1 = 40 

# vecteur des instants d’échantillonnage du signal modulé s_m
nu_mod = 1000
T_mod  = 1/nu_mod
N_mod  = int(1/(d_bin*T_mod))

tm = np.linspace(0, len(sb)/d_bin, len(sb)*N_mod)

# signal modulé

sm = []
for n in sb:
    if n == 0:
        nu = nu0
    elif n == 1:
        nu = nu1
    for k in range(N_mod):
        sm.append(A*cos(2*pi*nu*k*T_mod))
sm = np.array(sm)

In [ ]:
# Affichage de s_m
df_sm = pd.DataFrame({'x': tm , 'y': sm , 'legende': 'signal' })
fig3 = px.line(df_sm,x='x',  y='y', color='legende', labels={'x': 'tm' , 'y': 'sm'}, title= 'signal modulé' , width=700 )
fig3.show()

### Affichages

In [ ]:
# Affichage de s_b et s_m sur une seule figure
df_signals = pd.concat([df_sm,df_sb])
fig4 = px.line(df_signals, x='x',  y='y', color='legende', labels={'x':'Temps (s)', 'y':''}, title='Comparaison de sb et sm', width=700 )
fig4.show()

## Séance 2 - Estimation de la Densité De Probabilité (DDP)

### Analyses _in-silico_

In [ ]:
## Fonction de calcul d'histogramme
def histo(x,N=None,M=None):
    if N != None:
        x = x[0:N]
    else:
        N = len(x)
        
    if M == None:
        Delta = 3.49 * np.std(x) * N**(-1/3)
        M = round((max(x) - min(x))/Delta)
    else:
        Delta = (max(x) - min(x))/M

    hist, bin_edges = np.histogram(x, M)
    
    DDPest = hist / (Delta*N)
    c = (bin_edges[:-1] + bin_edges[1:])/2
        
    return DDPest, c, Delta

## Foncton d'affichage
def compareDDP(c, DDPest, DDPth, std, title=None):
    df_DDP_th_mean      = pd.DataFrame({'x':c,  'y':DDPth,  'legende':'DDP_th',    'line_dash':'solid','color':'red'})
    df_DDP_th_plus_std  = pd.DataFrame({'x':c,  'y':DDPth + std,  'legende':'DDP_th+std','line_dash':'dash', 'color':'black'})
    df_DDP_th_minus_std = pd.DataFrame({'x':c,  'y':DDPth - std,  'legende':'DDP_th-std','line_dash':'dash', 'color':'black'})
    df_DDP_th=pd.concat([df_DDP_th_mean,df_DDP_th_plus_std,df_DDP_th_minus_std])
    
    fig = px.line(df_DDP_th, x='x',  y='y', title=title,
                  color='legende', color_discrete_sequence=['orangered','black','black'], 
                  line_dash='legende', line_dash_sequence=['solid', 'dash', 'dash'])
    fig.add_bar(x=c, y=DDPest, marker_color='gray', name="DDP estimée")
    fig.show()

In [ ]:
# analyse du canal test
b = tsa.canal_test()

DDPest, c, Delta = histo(b, M = 20)

DDPth = np.exp(-(c**2)/2) / np.sqrt(2*pi)

N = len(b)
std = np.sqrt((DDPth/(N*Delta)) * (1 - DDPth*Delta))

compareDDP(c, DDPest, DDPth, std)

#### Influence de N

In [ ]:
N=[100, 1000, 10000, 100000]
M = 20

for n in N:
    make_subplots()

    b = tsa.canal_test()

    DDPest, c, Delta=histo(b, n, M)
    
    DDPth = np.exp(-(c**2)/2) / np.sqrt(2*pi) #Gaussienne
    std = np.sqrt((DDPth/(n*Delta)) * (1 - DDPth*Delta))

    compareDDP(c, DDPest, DDPth, std, "N = "+str(n)) #Comparer avec std gaussienne


#### Influence de $\Delta$

In [ ]:
N = 1000
M=[10, 20, 1000]

for m in M:
    make_subplots()

    b = tsa.canal_test()

    DDPest, c, Delta=histo(b, N, m)

    DDPth = np.exp(-(c**2)/2) / np.sqrt(2*pi) #Gaussienne - bien mettre avec x et non DDpest
    std=np.sqrt((DDPth/(N*Delta))*(1-DDPth*Delta))

    compareDDP(c, DDPest, DDPth, std, "Delta = "+str(Delta)) #Comparer avec std gaussienne


### Analyses _in-situ_

In [ ]:
# analyse du canal de transmission réel
canal_id = 5
sm_tr = tsa.transmit(sm , canal_id)

DDPest, c, Delta = histo(sm_tr, 10000, 20)

fig = go.Figure()
fig.add_bar(x=c, y=DDPest, marker_color='gray', name="DDP estimée")
fig.show()

## Séance 3 - Estimation de la Densité Spectrale de Puissance Moyenne

### Analyses _in-silico_

In [ ]:
def compareDSP(f, fth,DSPth,DSPbiais,DSPest):
    """Fonction d’affichage compareDSP(f, DSPth, DSPbiais, DSPest) commune aux trois estimateurs, qui trace en dB sur un même graphe.
    Args:
        f (_type_): vecteur de fréquences réduites tel que 0 ≤ f < 0.5
        DSPth (_type_): la DSPM théorique vraie du canal test ΓX (f),
        DSPbiais (_type_): la DSPM théorique obtenue en utilisant l’estimateur ??? du canal test ΓX ∗W_{N,B}(f),
        DSPest (_type_): votre DSPM estimée dΓ_{???}(f),
    """
    df_DSPth      = pd.DataFrame({'x':fth,  'y':DSPth,  'legende':'DDP_th',    'line_dash':'solid','color':'red'})
    df_DSPbiais  = pd.DataFrame({'x':fth,  'y':DSPbiais,  'legende':'DSPbiais','line_dash':'solid', 'color':'blue'})
    df_DSPest = pd.DataFrame({'x':f,  'y':DSPest,  'legende':'DSPest','line_dash':'solid', 'color':'black'})
    df_DSP=pd.concat([df_DSPth,df_DSPbiais,df_DSPest])
    
    fig = px.line(df_DSP, x='x',  y='y',
                  color='legende', color_discrete_sequence=['orangered','blue','black'], 
                  line_dash='legende', line_dash_sequence=['solid', 'solid', 'solid'])
    fig.show()

##### Estimateur spectral simple

In [ ]:
def estimateur_simple(x, nd, N, nfft):
    Xest = x[nd:(nd+N)]
    fftest = fft(Xest, nfft)
    DSPest1 = (1/N) * (abs(fftest)**2)
    DSPest1 = 10*np.log10(DSPest1)
    f = np.linspace(0, 1-(1/nfft), nfft) #frequence réduite allant de 0 à 1 avec un pas égal à  1/nfft
    return DSPest1,f

##### Estimateur spectral moyenné

In [ ]:
def estimateur_moyenne(x, M, nfft):
    f, DSPest2 = signal.welch(x, nperseg=M, nfft=nfft, window='boxcar',noverlap=0)
    DSPest2 = 10 * np.log10(DSPest2)
    return DSPest2,f

##### Estimateur spectral de Welch

In [ ]:
def estimateur_welch(x, window, M, Noverlap, nfft):
    f, DSPest3 = signal.welch(x,window=window, nperseg=M, noverlap=Noverlap, nfft=nfft)
    DSPest3 = 10 * np.log10(DSPest3)
    return DSPest3,f

In [ ]:
b = tsa.canal_test(filtered=True)

nd = 0
N = 10000
nfft = 1024
DSPest1,f1 = estimateur_simple(b, nd, N, nfft)
DSPth1, DSPbiais1, f1th = tsa.sptheo(N, 'simple')

compareDSP(f1, f1th, DSPth1,DSPbiais1, DSPest1)

M= 1000
nfft = 1024
DSPest2,f2 = estimateur_moyenne(b, M, nfft)
DSPth2, DSPbiais2, f2th = tsa.sptheo(M, 'moyenne')

compareDSP(f2, f2th, DSPth2,DSPbiais2, DSPest2)

window = 'hann'
M= 1000
Noverlap = 3*M/4
nfft = 1024
DSPest3,f3 = estimateur_welch(b, window, M, Noverlap, nfft)
DSPth3, DSPbiais3, f3th = tsa.sptheo(M, 'welch', window)

compareDSP(f3, f3th, DSPth3,DSPbiais3, DSPest3)

### Analyses _in-situ_

In [ ]:
# analyse du canal de transmission réel
sm_tr = tsa.transmit(sm , canal_id)

window = 'hann'
M= 1000
Noverlap = 3*M/4
nfft = 1024
DSPest,f = estimateur_welch(sm_tr, window, M, Noverlap, nfft)

df_DSPest = pd.DataFrame({'x':f,  'y':DSPest,  'legende':'DSPest','line_dash':'solid', 'color':'black'})
fig = px.line(df_DSPest, x='x',  y='y', color='legende', line_dash='legende')
fig.show()


## Séance 4 - Détection d’un signal noyé dans un bruit

In [ ]:
# Transmission
sm_tr = tsa.transmit(sm, canal_id,powB=0.0001)

#### Débruitage par détection

In [ ]:
k = 1
nfft = len(sm_tr)+N_mod-1
f = np.fft.rfftfreq(nfft)
Gamma = 1/np.where(f==0,float('inf'),f)

# Gain complexe des filtres de détection
t= np.linspace(0, N_mod,N_mod)
s0 = A*np.cos(2*pi*nu0*t*T_mod)
H0 = k * (np.fft.rfft(s0, nfft)/np.where(Gamma==0,float('inf'),Gamma)) #* np.exp(2j * pi * f/nu0)
s1 = A*np.cos(2*pi*nu1*t*T_mod)
H1 = k * (np.fft.rfft(s1, nfft)/np.where(Gamma==0,float('inf'),Gamma)) #* np.exp(2j * pi * f/nu1)
S = np.fft.rfft(sm_tr, nfft)
# signaux en sortie des 2 filtres
s0 = np.fft.irfft(H0 * S, nfft)
s1 = np.fft.irfft(H1 * S, nfft)

sd = []

for n in range(N_mod - 1, nfft, N_mod):
    sd.append(int(s0[n] < s1[n]))
sd=np.array(sd)
th = np.linspace(0, len(sd)/d_bin, len(sd))

dsd      = pd.DataFrame({'x':tb,  'y':sd,  'legende':'sd',    'line_dash':'solid','color':'red'})
dsb  = pd.DataFrame({'x':tb,  'y':sb,  'legende':'sb','line_dash':'solid', 'color':'blue'})
df_DSP=pd.concat([dsd,dsb])
    
fig = px.line(df_DSP, x='x',  y='y',
                color='legende', color_discrete_sequence=['orangered','blue'], 
                line_dash='legende', line_dash_sequence=['solid', 'solid'])
fig.show()


#### Étude de l’effet du bruit de transmission sur la précision

In [ ]:
def error(sd,sb):
    pass
    return e



#### Décodage

## Séance 5 - Prédiction AR d’ordre $\textit{M}$

#### Estimation de la fonction d’autocorrélation

In [ ]:
s_tr=s
K = 200

Gam = signal.correlate(s_tr.astype(np.float32), s_tr.astype(np.float32), mode='full', method='auto')
Gam /= len(s_tr)
Gam = Gam[len(Gam)//2 - K:len(Gam)//2 + K + 1]
lags = signal.correlation_lags(len(s_tr), len(s_tr), mode="full")
lags = lags[len(lags)//2 - K:len(lags)//2 + K + 1]

df_G = pd.DataFrame({'x': lags , 'y': Gam , 'legende':'Gam'})
fig = px.line(df_G, x='x',  y='y', color='legende', labels={'x':'Temps (s)', 'y':'Gam'}, title='fonction d’autocorrélation', width=700 )
fig.show()

#### Identification du modèle AR(M)

In [ ]:
M = 16

# matrice du systeme lineaire
G1 = Gam[200:(200+M+1)]
G = toeplitz(G1)

# second membre du systeme lineaire
b = [0]*(M+1)
b[0] = -1

# solution du systeme G.Phi = b
Phi = np.linalg.pinv(G) @ b

# coefficients du filtre de Wiener
sigma2 = - 1/ Phi[0] #sigma au carré
h = sigma2 * Phi[1:]

# puissance de l'erreur
sigma = np.sqrt(sigma2)

#### Prédiction Linéaire

In [ ]:
s_hat = []

for n in range(len(s_tr)):
    nbr = 0
    for k in range(1, M+1):
        if(n-k>=0):
            nbr += h[k-1]*s_tr[n-k]
    s_hat.append(nbr)

N = len(s_tr) # nombre de points du signal s
D = N/Fs # durée du signal s
ts = np.linspace(0, D, N)

ds_tr      = pd.DataFrame({'x':ts,  'y':s_tr,  'legende':'s',    'line_dash':'solid','color':'red'})
ds_hat  = pd.DataFrame({'x':ts,  'y':s_hat,  'legende':'s_hat','line_dash':'solid', 'color':'blue'})
df_DSP=pd.concat([ds_tr,ds_hat])
    
fig = px.line(df_DSP, x='x',  y='y',
                color='legende', color_discrete_sequence=['orangered','blue'], 
                line_dash='legende', line_dash_sequence=['solid', 'solid'])
fig.show()

E = np.abs(s_hat - s_tr)
ds_E = pd.DataFrame({'x':ts,  'y':E,  'legende':'E'})
fig = px.line(ds_E, x='x',  y='y', line_dash='legende')
fig.show()

#### Restauration par filtrage causal/anti-causal

In [ ]:
s_anticausale = []

for n in range(len(s_tr)):
    nbr = 0
    for k in range(1, M+1):
        if n + k < len(s_tr):
          nbr += h[k-1]*s_tr[n+k]
    s_anticausale.append(nbr)

ds_tr      = pd.DataFrame({'x':ts,  'y':s_tr,  'legende':'s',    'line_dash':'solid','color':'red'})
ds_anticausale = pd.DataFrame({'x':ts,  'y':s_anticausale,  'legende':'s_anticausale','line_dash':'solid', 'color':'blue'})
df_DSP=pd.concat([ds_tr,ds_anticausale])
    
fig = px.line(df_DSP, x='x',  y='y',
                color='legende', color_discrete_sequence=['orangered','blue'], 
                line_dash='legende', line_dash_sequence=['solid', 'solid'])
fig.show()

E_anticausale = np.abs(s_anticausale - s_tr)
dE_anticausale = pd.DataFrame({'x':ts,  'y':E_anticausale,  'legende':'E_anticausale'})
fig = px.line(dE_anticausale, x='x',  y='y', line_dash='legende')
fig.show()

In [ ]:
s_filtre = []
for i in range(len(E)):
   if (E[i]>7) and (E_anticausale[i]>7):
      s_filtre.append(s_hat[i])
   else:
      s_filtre.append(s_tr[i])

ds_tr = pd.DataFrame({'x':ts,  'y':s_tr,  'legende':'s_tr',    'line_dash':'solid','color':'red'})
ds_filtre = pd.DataFrame({'x':ts,  'y':s_filtre,  'legende':'s_filtre','line_dash':'solid', 'color':'blue'})
df_DSP=pd.concat([ds_tr,ds_filtre])
    
fig = px.line(df_DSP, x='x',  y='y',
                color='legende', color_discrete_sequence=['orangered','blue'], 
                line_dash='legende', line_dash_sequence=['solid', 'solid'])
fig.show()
